In [18]:
import pandas as pd
import math

def calculate_saturation_pressure_water(t2m):
    a1 = 611.21
    a3 = 17.502
    a4 = 32.19
    T0 = 273.16
    return a1 * t2m.apply(lambda x: math.exp(a3 * (x - T0) / (x - a4)))

def calculate_saturation_pressure_ice(t2m):
    a1 = 611.21
    a3 = 22.587
    a4 = -0.7
    T0 = 273.16
    return a1 * t2m.apply(lambda x: math.exp(a3 * (x - T0) / (x - a4)))

def calculate_relative_humidity(t2m, p, q):
    epsilon = 0.621981
    esat_w = calculate_saturation_pressure_water(t2m)
    esat_i = calculate_saturation_pressure_ice(t2m)
    alpha = calculate_mixed_phase_parameter(t2m)
    esat = alpha * esat_w + (1 - alpha) * esat_i
    e = (p * q ** (1 / epsilon)) / (esat * (1 + q * (1 / epsilon - 1)))
    RH = 100 * e
    return RH

def calculate_mixed_phase_parameter(t2m):
    Tice = 250.16
    T0 = 273.16
    return t2m.apply(lambda x: 0 if x <= Tice else (1 if x >= T0 else ((x - Tice) / (T0 - Tice)) ** 2))

def calculate_specific_humidity(t2m, p):
    Rdry = 287.05  # Specific gas constant for dry air in J/(kg·K)
    Rvap = 461.5   # Specific gas constant for water vapor in J/(kg·K)
    esat = calculate_saturation_pressure_water(t2m)
    q_sat = ((Rvap / Rdry) * esat) / (p - (1 - Rvap / Rdry) * esat)
    return q_sat

# Load the CSV file into a DataFrame
df = pd.read_csv('D:/ESMI_Dataset/Final_Data/Panel_Data/panel_data_240417_test.csv')

# Convert T to t2m (assuming T represents air temperature in Celsius)
df['t2m'] += 273.15

# Calculate specific humidity for each row
df['Specific Humidity'] = calculate_specific_humidity(df['t2m'], df['sp'])

# Calculate relative humidity for each row
df['Relative Humidity'] = calculate_relative_humidity(df['t2m'], df['sp'], df['Specific Humidity'])

# Save the DataFrame back to CSV
df.to_csv('your_modified_file.csv', index=False)
